# 데이터합치기
### 지하철데이터 + 요일, 공휴일 + 확진자데이터 + 검색어트랜드

In [2]:
import pickle
import pandas as pd
from function import makefulldate, makefulldate2
from function import DFyoon

In [3]:
dfyoon = DFyoon()
len(dfyoon.station_list)

불러오기완료


250

In [16]:
df = pd.read_pickle('data/8주평균하차수.pkl')

In [15]:
df = pd.read_pickle('data/지하철데이터.pkl')
df.reset_index(inplace=True)
df.rename(columns={'날짜':'date'}, inplace=True)
col_need = ['date','역명', '5-6', '6-7', '7-8', '8-9', '9-10', '10-11', '11-12',
       '12-13', '13-14', '14-15', '15-16', '16-17', '17-18', '18-19', '19-20',
       '20-21', '21-22', '22-23', '23-24', '24-01']
col_nee= ['5-6', '6-7', '7-8', '8-9', '9-10', '10-11', '11-12',
       '12-13', '13-14', '14-15', '15-16', '16-17', '17-18', '18-19', '19-20',
       '20-21', '21-22', '22-23', '23-24', '24-01']

In [4]:
len(df)

284560

In [17]:
#역명이 바뀌거나 중복된 것 modify
renamed_dic = {'서울역':'서울',
               "낙성대(강감찬)":'낙성대',
               '용마산(용마폭포공원)':'용마산',
               '동대문역사문화공원(DDP)':'동대문역사문화공원'}
for i in range(len(df['역명'])):
    if df['역명'][i] in ['서울역','낙성대(강감찬)','용마산(용마폭포공원)','동대문역사문화공원(DDP)']:
        
        df['역명'][i] = renamed_dic[df['역명'][i]]
df = df.query('역명 not in ["신내"]')
df.to_pickle('data/전년유동감소폭.pickle')
df_date2 = makefulldate2()
df = pd.merge(df_date2,df, on = 'date',how='left')


C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [18]:
df_down_ = df[df['구분']=='하차']
df_down_ = df_down_[col_need+['요일']].groupby(['역명','요일','date']).sum()

import numpy as np
from copy import deepcopy
df_down_temp = deepcopy(df_down_)

In [19]:
df_down_temp.reset_index(inplace=True)
df_down_temp.set_index(['역명','요일','date'],inplace=True)

# 지하철 몇주전 평균낼껀지 lastnday에 넣는다

In [20]:
#지난 8주 평균
lastnday=8

In [21]:
from tqdm import tqdm
for i in tqdm(df_down_temp.reset_index()['역명'].unique()):
    try:
        for j in df['요일'].unique():
            for k,val in enumerate(df_down_.loc[i,j].index):
                if k>7:
                    df_down_temp.loc[(i,j,val)] = np.mean(df_down_.loc[i,j].iloc[k-lastnday:k])
                else :
                    df_down_temp.loc[i,j].iloc[k] = df_down_.loc[i,j].iloc[k]
    except:
        print(i,j,val)

100%|████████████████████████████████████████████████████████████████████████████████| 242/242 [01:00<00:00,  3.99it/s]


In [22]:
new_df_ = pd.DataFrame(df_down_temp.reset_index().drop(['요일'],axis=1).set_index(['역명','date']).stack())
new_df_= new_df_.reset_index()
new_df_.columns = ['역명','date','시간','8주평균하차수']

In [23]:
new_df_

,역명,date,시간,8주평균하차수
0,가락시장,2019-12-06,5-6,95.000
1,가락시장,2019-12-06,6-7,409.000
2,가락시장,2019-12-06,7-8,912.000
3,가락시장,2019-12-06,8-9,3065.000
4,가락시장,2019-12-06,9-10,1526.000
...,...,...,...,...
885715,효창공원앞,2020-05-26,20-21,270.250
885716,효창공원앞,2020-05-26,21-22,243.250
885717,효창공원앞,2020-05-26,22-23,213.750
885718,효창공원앞,2020-05-26,23-24,133.375


# 승차수, 하차수, 요일 공휴일 코로나단계 합치기

In [27]:
df_date = makefulldate()
df = pd.merge(df_date,df, on = 'date',how='left')
##요일 공휴일 데이터프레임화
df_yo1 = df[['date','요일','공휴일']].drop_duplicates()
df_yo = df[['date','요일','공휴일']].drop_duplicates()
df_yo['주말제외공휴일'] = df['공휴일']

df_down = df[df['구분']=='하차']
df_down.drop(columns=['호선','역번호','합 계'], inplace =True)
df_1 = df_down[col_need].groupby(['역명','date']).sum()

df_up = df[df['구분']=='승차']
df_up.drop(columns=['호선','역번호','합 계'], inplace =True)
df_2 = df_up[col_need].groupby(['역명','date']).sum()

new_df = pd.DataFrame(df_1.stack())
new_df.reset_index(inplace = True)
new_df.columns=['역명','date','시간','하차']

new_df['승차']=df_2.stack().values

do = pd.read_csv('data/feature_list_dy.csv',parse_dates=['date'])
new_df = pd.merge(new_df,df_yo,on = 'date',how='left')#######################################
new_df = pd.merge(new_df,do[['date','corona_stage']],on = 'date',how='left')##################
new_df['구'] = [1 if dfyoon.station_gu_dic[i] in dfyoon.gu_list else 0 for i in new_df['역명'] ]
renew_df = new_df[new_df['구']==1]

C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [29]:
renew_df = pd.merge(renew_df, new_df_, on=['역명','date','시간'], how='left')

---

# 코로나 감염자데이터 지난 last_nday 데이터 합치기

In [31]:
for i,val in enumerate(renew_df['역명'].unique()):
    df_sta = dfyoon.sta_merg(val, last_nday=7)
    df_sta['역명']=val
    if i==0:
        df_conc = df_sta
    else :
        df_conc = pd.concat([df_conc,df_sta],axis=0)
    ra = pd.merge(renew_df,df_conc, on = ['date','역명'],how='left')
ra.drop('구', axis=1, inplace=True)

C:\Users\YOON\nlp\worksp\re\big_contest\modeling\function.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sumcount"][i] = sum(df[col_name][:i])
C:\Users\YOON\nlp\worksp\re\big_contest\modeling\function.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sumcount"][i] = sum(df[col_name][i - last_nday : i])


# 한양대기준트렌드

In [48]:
import pandas as pd
df_han = pd.read_pickle('data/검색어트렌드_한양대_final.pkl')
df_han.set_index('date', inplace=True)

In [49]:
df_han = df_han.stack().reset_index()
df_han.columns=['date','역명','검색트랜드(한양)']
df_han['date'] = pd.to_datetime(df_han['date'])
df_han['역명'] = [i[:-1] for i in df_han['역명']]
df_merged = pd.merge(ra, df_han, on = ['date','역명'],how='left')

In [50]:
import re
df_merged['시간'] =[int(re.findall('(.*)-',i)[0]) for i in df_merged['시간']] 

In [51]:
df_merged['시간'] =[int(i) for i in df_merged['시간']] 

# 기상데이터

In [52]:
df_wea = pd.read_pickle('data/날씨(기상자료개방포털).pkl')
df_wea.head()

,date,시간,기온(°C),강수량(mm),풍속(m/s),습도(%)
0,2020-01-01,0,-6.5,0.0,0.0,38
1,2020-01-01,1,-5.9,0.0,1.7,40
2,2020-01-01,2,-5.7,0.0,0.1,42
3,2020-01-01,3,-5.6,0.0,0.0,46
4,2020-01-01,4,-5.4,0.0,0.0,50


In [53]:
df_wea['date'] = pd.to_datetime(df_wea['date'])
df_wea['시간'].astype(int)
df_wea['시간'] = [24 if i==0 else i for i in df_wea['시간'] ]
from dateutil.relativedelta import relativedelta
df_wea['date'] = [i - relativedelta(days=1) if df_wea['시간'][idx]==24 else i for idx, i in enumerate(df_wea['date'])]

In [54]:
df_wea

,date,시간,기온(°C),강수량(mm),풍속(m/s),습도(%)
0,2019-12-31,24,-6.5,0.0,0.0,38
1,2020-01-01,1,-5.9,0.0,1.7,40
2,2020-01-01,2,-5.7,0.0,0.1,42
3,2020-01-01,3,-5.6,0.0,0.0,46
4,2020-01-01,4,-5.4,0.0,0.0,50
...,...,...,...,...,...,...
5851,2020-08-31,19,28.0,0.0,3.5,73
5852,2020-08-31,20,27.6,0.0,2.5,73
5853,2020-08-31,21,27.1,0.0,2.4,74
5854,2020-08-31,22,26.8,0.0,2.3,76


In [55]:
df_merged = pd.merge(df_merged, df_wea, on=['date','시간'], how='left')

In [65]:
df_merged.to_pickle('data/merged_dataframe.pickle')